<a href="https://colab.research.google.com/github/chonorat/Classification/blob/master/Bagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import sys
import random
import pandas as pd
import numpy as np
##Dot Product
def dot_prod(W_Vector,F_Vector):
        dp=0
        for i in range(0,len(W_Vector)):
                dp=dp+(W_Vector[i]*(F_Vector[i]))
        return(dp)

##Read Data
datafile='/content/ionosphere.data'
f=open(datafile)
data=[]
i=0
l=f.readline()
while(l!=''):
        a=l.split()
        l2=[]
        for j in range(0,len(a),1):
                l2.append(float(a[j]))
        data.append(l2)
        l=f.readline()
rows=len(data)
cols=len(data[0])

##Read Labels
labelfile='/content/ionosphere_trailabels.data'
f=open(labelfile)
trainlabels={}
n=[]
n.append(0)
n.append(0)
l=f.readline()
while(l !=''):
        a=l.split()
        if int(a[0])==1 or int(a[0])==0:
                trainlabels[int(a[1])]=int(a[0])
                n[int(a[0])]+=1
        l=f.readline()

##set class labels
for i in range(0,rows-1):
        if(trainlabels.get(i)!=None and trainlabels[i]==0):
                trainlabels[i]=-1

data=np.array(data)

##Train data count
def traincoun(samplelength,sample_index):
        traincount=0
        for i in range(0,samplelength):
                p=sample_index[i]
                if trainlabels.get(p)!=None:
                        traincount=traincount+1
        return traincount

##split on value
def split_col(split,column,rows,sample_index):
        set1=[]
        set2=[]
        for i in range(0,rows):
                l=sample_index[i]
                if(trainlabels.get(l)!=None):
                        if data[l][column]<split:
                                set1.append(trainlabels[l])
                        else:
                                set2.append(trainlabels[l])
        return set1,set2
##Count -1 labels in set
def countX(lst):
    count = 0
    for ele in lst:
        if (ele == -1):
            count = count + 1
    return count


##CART
def cart(sample_index,data,trainlabels,col,traincount):
        rows=len(sample_index)
        gini,best_gini,best_col,best_thresh,bottom=10000000,10000,10000,10000,100000
        for k in range(0,cols):
                for i in range(0,rows):
                        j=sample_index[i]
                        set1,set2=split_col(data[j][k],k,rows,sample_index)
                        lsize,rsize=len(set1),len(set2)
                        try:
                                lp=countX(set1)
                                rp=countX(set2)
                                gini = (lsize/traincount)*(lp/lsize)*(1 - lp/lsize) + (rsize/traincount)*(rp/rsize)*(1 - rp/rsize)
                        except:
                                pass
                        if gini<best_gini:
                                best_gini=gini
                                best_thresh=data[sample_index[i]][k]
                                best_col=k
                                try:
                                        bottom=min(data[data[:,k]<data[j,k],k], key=lambda x:data[j,k]-x)
                                except:
                                        pass
        try:
                best_thresh=(best_thresh+bottom)/2
        except:
                pass
        lab1=np.argmax(set1.count(1))
        lab2=np.argmax(set2.count(1))
        if lab1/len(set1)>lab2/len(set2):
                lab1=1
                lab2=0
        else:
                lab1=0
                lab2=1
        return(best_col,best_thresh,lab1,lab2)
      
def sampl(rows):
        rand_sample=[]
        leng=np.round(rows*.63)
        for x in range(0,int(leng)):
                rand_sample.append( random.randint(1,rows-1))
        #rand_samp=random.sample(xrange(351), 351)
        return rand_sample

def classify(data,trainlabels,thresholds):
        rows=len(data)
        pred=[]
        for i in range(0,rows):
                to_append=[]
                if(trainlabels.get(i)==None):
                        for j in range(0,len(thresholds)):
                                split_col=thresholds[j][0]
                                split_val=thresholds[j][1]
                                lab1=thresholds[j][2]
                                lab2=thresholds[j][3]
                                if data[i][split_col]>split_val:
                                        to_append.append(lab1)
                                else: to_append.append(lab2)
                        print(np.argmax(np.bincount(to_append)),i)
                        pred.append(to_append)
        return pred

##Main_Method
thresholds=[]
for i in range(0,10):
        samplelength=int(np.round(rows*.63))
        sample_index=np.random.choice(len(data),int(np.round( len(data)*.63)), replace=True)
        traincount=traincoun(samplelength,sample_index)
        col,thresh,lab1,lab2=cart(sample_index,data,trainlabels,cols,traincount)
        thresholds.append([col,thresh,lab1,lab2])
print(thresholds)
pred=classify(data,trainlabels,thresholds)


[[6, 0.016989999999999998, 0, 1], [4, 0.15603, 0, 1], [4, 0.15603, 0, 1], [4, 0.041440000000000005, 0, 1], [26, 0.999945, 0, 1], [4, 0.23723, 0, 1], [4, 0.145975, 0, 1], [2, 0.19028, 0, 1], [4, 0.041440000000000005, 0, 1], [4, 0.23723, 0, 1]]
0 3
1 5
1 21
0 31
0 39
0 56
0 63
0 88
0 89
0 99
0 103
1 106
0 137
0 151
1 156
0 167
0 177
0 181
1 190
0 210
0 216
0 221
0 234
0 239
0 251
0 254
0 262
0 276
0 288
0 292
0 298
0 300
0 307
0 337
0 343
0 349
